# 1. Downloading dependencies

In [ ]:
# Installing PySpark 
!pip install pyspark

In [ ]:
!pip install findspark

In [ ]:
import findspark
findspark.init()

In [ ]:
import sys
import boto3
from awsglue.job import Job
from awsglue.transforms import *
from pyspark import SparkContext
from pyspark.sql import SparkSession
from awsglue.context import GlueContext
from awsglue.utils import getResolvedOptions
from awsglue.dynamicframe import DynamicFrame

## 1.1 Creating GlueContext and loading data

Firstly I'm creating a Spark Dataframe to convert in a Glue Dynamic DF later.

In [ ]:
spark = SparkSession.builder.master("local[*]").appName("trips_data").getOrCreate()

In [ ]:
spark

In [ ]:
default_path = "s3://903442739132-source-bucket-trips-data/"

In [ ]:
df = spark.read.parquet(f"{default_path}/fhvhv_tripdata_2021-*.parquet")

In [ ]:
df.rdd.getNumPartitions()

In [ ]:
df.printSchema()

In [ ]:
glueContext = GlueContext(spark.sparkContext)

In [ ]:
df_dynamic = DynamicFrame.fromDF(df, glueContext, "df_dynamic")

# 2. Separating files by type of license

In [ ]:
from pyspark.sql.functions import col

In [ ]:
df_uber = df_dynamic.filter(f=lambda x: x["hvfhs_license_num"] in "HV0003")

In [ ]:
s3 = boto3.resource("s3")
bucket = "903442739132-type-of-license-bucket"
filename = "uber_data.csv"

In [ ]:
connection_options = {"path": "s3://903442739132-type-of-license-bucket/"}

In [ ]:
df_uber.write(connection_type="s3", connection_options=connection_options)